# Data Science Master Class

organizowany przez DataWorkschop

Rozwiązanie w konkursie Kaggle
Predykcja cen mieszkań

## Imporujemy biblioteki

In [41]:
import pandas as pd
import numpy as np
import helper as h
import eli5

import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error


### Wczytujemy i łączymy dane

In [2]:
df_train = pd.read_hdf("../input/train_data.h5")
df_train['price'] = df_train['price'].map(h.parse_price)

df_test = pd.read_hdf("../input/test_data.h5")

df = pd.concat([df_train, df_test])
print(df.shape, df.shape)

(34180, 8) (34180, 8)


### Params

In [3]:
params = df["params"].apply(pd.Series)
params = params.fillna(-1)

if "Охрана:" not in df:
    df = pd.concat([df, params], axis=1)
    
    obj_feats = params.select_dtypes(object).columns

    for feat in obj_feats:
        df["{}_cat".format(feat)] = df[feat].factorize()[0]

cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats

['Охрана:_cat',
 'Тип здания:_cat',
 'Тип объекта:_cat',
 'Количество корпусов:_cat',
 'Тип объявления:_cat',
 'Застройщик:_cat',
 'Общая площадь:_cat',
 'Дата публикации:_cat',
 'Количество комнат:_cat',
 'Парковка:_cat',
 'Дата  обновления:_cat',
 'Количество этажей:_cat',
 'Сдача:_cat',
 'Комиссия агенту:_cat',
 'Высота потолков:_cat',
 'Этаж:_cat',
 'Этап строительства:_cat',
 'Новостройка:_cat',
 'Количество квартир:_cat',
 'Класс жилья:_cat',
 'Адрес:_cat',
 'Лифт:_cat',
 'Вид из окна:_cat',
 'Возможна ипотека:_cat',
 'Год постройки:_cat',
 'Этажность:_cat',
 'Мусоропровод:_cat',
 'Ремонт:_cat',
 'Площадь кухни:_cat',
 'Жилая комната:_cat',
 'Тип санузла:_cat',
 'Тип балкона:_cat',
 'Мебель на кухне:_cat',
 'Холодильник:_cat',
 'Интернет:_cat',
 'Мебель:_cat',
 'Телефон:_cat',
 'Свободная планировка:_cat',
 'Управляющая компания:_cat',
 'Количество подъездов:_cat',
 'Тип дома:_cat',
 'Тип комнат:_cat',
 'Покрытие пола:_cat',
 'Серия:_cat',
 'Тип перекрытия:_cat',
 'Возможен торг:

### Powierzchnia całkowita 

In [4]:
def parse_area(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return float(val.split("м")[0].replace(" ", ""))

df["area"] = df["Общая площадь:"].map(h.parse_area)
df["kitchen_area"] = df["Площадь кухни:"].map(parse_area)

### Sprawdzamy

Dodajmy dwie funkcje  - `check_model` i `check_log_model`. 

In [38]:
def check_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()
    df_test = df[ df["price"].isnull() ].copy()

    X_train = df_train[feats]
    y_train = df_train["price"]
    
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)
    return np.mean(scores), np.std(scores)


def check_log_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()

    X = df_train[feats]
    y = df_train["price"]
    y_log = np.log(y)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

        
        #model = xgb.XGBRegressor(max_depth=5, n_estimators=50, random_state=0)
        model=model
        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = mean_absolute_error(y_test, y_pred)
        scores.append(score)

    return np.mean(scores), np.std(scores), model

## Lokalizacja 

Spróbujmy podziałać z  `geo_block` i `breadcrumbs`.

In [6]:
df["geo_block"].values

array([list(['г. Москва', 'Лианозово', 'г. Москва', 'Лианозово']),
       list(['г. Москва', 'Ховрино', 'г. Москва', 'Ховрино']),
       list(['г. Москва', 'ул Лобачевского', 'г. Москва', 'ул Лобачевского']),
       ..., list(['г. Москва', 'Ховрино', 'г. Москва', 'Ховрино']),
       list(['Новая Москва', 'п. Внуковское', 'г. Москва', 'Новая Москва', 'п. Внуковское', 'г. Москва']),
       list(['г. Москва', 'Ярославский', 'ул Красная Сосна', 'г. Москва', 'Ярославский', 'ул Красная Сосна'])],
      dtype=object)

Z jakiegoś powodu  w `geo_blocks` są widoczne duplikaty. Na przykład w pierwszym wierszu mamy dwa elementy powtórzone dwukrotnie.

Tak to teraz wygląda:
`['г. Москва', 'Лианозово', 'г. Москва', 'Лианозово']`.

In [7]:
df["geo_block_norm"] = df["geo_block"].map(lambda x: x[:int(len(x)/2)])

df["geo_block_norm"]

0                            [г. Москва, Лианозово]
1                              [г. Москва, Ховрино]
2                      [г. Москва, ул Лобачевского]
3                  [г. Москва, Ховрино, ул Дыбенко]
4                          [г. Москва, Даниловский]
                            ...                    
11443                                   [г. Москва]
11444                [г. Москва, Хорошёво-Мнёвники]
11445                          [г. Москва, Ховрино]
11446      [Новая Москва, п. Внуковское, г. Москва]
11447    [г. Москва, Ярославский, ул Красная Сосна]
Name: geo_block_norm, Length: 34180, dtype: object

Ta konstrukcja `x[:int(len(x)/2)]`  może okazać się mało zrozumiałą. 

Podejdzmy do tego przykładu w prostszy sposób.  Stwórzmy listę 3 elementów i pokażmy tylko dwa pierwsze.

In [8]:
x = [1, 2, 3, 4]
x[0:2]

[1, 2]

Zero, które jest z przodu można pominąć, to będzie dawało ten sam wynik i wtedy będzie to wyglądało tak: 


In [9]:
x[:2]

[1, 2]

Sprawdźmy teraz długość listy (ile elementów jest w liście).


In [10]:
len(x)

4

Jeśli chcemy sprawdzić, ile to połowa, można zrobić to tak: 

In [11]:
len(x) / 2

2.0

Tylko dostaliśmy `float`, czyli wartość zmiennoprzecinkowa. Dlatego musisz przekonwertować ją na liczbę całkowitą - to jest `int`.


In [12]:
int(len(x) / 2)

2

Zbieramy wszystko razem. 

In [13]:
x[:int(len(x)/2)]

[1, 2]

Idziemy dalej i tworzymy  `DataFrame`.

In [14]:
geo_block = (
df["geo_block"]
    .map(lambda x: x[:int(len(x)/2) ])
    .map(lambda x: {"geo_block_{}".format(idx):val for idx,val in enumerate(x) })
    .apply(pd.Series)
)

geo_block

,geo_block_0,geo_block_1,geo_block_2,geo_block_3,geo_block_4,geo_block_5
0,г. Москва,Лианозово,NaN,NaN,NaN,NaN
1,г. Москва,Ховрино,NaN,NaN,NaN,NaN
2,г. Москва,ул Лобачевского,NaN,NaN,NaN,NaN
3,г. Москва,Ховрино,ул Дыбенко,NaN,NaN,NaN
4,г. Москва,Даниловский,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
11443,г. Москва,NaN,NaN,NaN,NaN,NaN
11444,г. Москва,Хорошёво-Мнёвники,NaN,NaN,NaN,NaN
11445,г. Москва,Ховрино,NaN,NaN,NaN,NaN
11446,Новая Москва,п. Внуковское,г. Москва,NaN,NaN,NaN


Okazuje się, że maksymalna „głębokość” to 5. Możesz również zobaczyć, jakie wartości są w środku i natychmiast uruchomić również `.factorize()`.


In [15]:

for feat in geo_block.columns:
    print(feat)
    print(geo_block[feat].value_counts().head(20))
    print("="*100)
    print()
    

    df["{}_cat".format(feat)] = geo_block[feat].factorize()[0]

geo_block_0
г. Москва             29049
Новая Москва           4531
г. Зеленоград           318
п. Свиблово             138
п. Некрасовка           134
п. Новобратцевский        3
п. Крюково                1
п. Алабушево              1
п. Главмосстроя           1
пгт Акулово               1
д. Сколково               1
п. Подушкино              1
п. Рублево                1
Name: geo_block_0, dtype: int64

geo_block_1
п. Сосенское              2522
Хорошёво-Мнёвники         1142
Филёвский Парк             945
п. Московский              919
Ховрино                    829
Раменки                    811
Люблино                    666
Некрасовка                 654
Кунцево                    537
Орехово-Борисово Южное     534
ул Дыбенко                 525
Выхино-Жулебино            496
Зябликово                  492
ул Лобачевского            442
п. Внуковское              430
Рязанский                  425
Марьино                    407
Пресненский                378
Лефортово            

In [16]:
geo_cat_feats = [x for x in df.columns if "geo_block" in x and "_cat" in x]
geo_cat_feats

['geo_block_0_cat',
 'geo_block_1_cat',
 'geo_block_2_cat',
 'geo_block_3_cat',
 'geo_block_4_cat',
 'geo_block_5_cat']

### Sprawdzamy wynik 

Sprawdzamy wynik, dodając nowe funkcje.


In [17]:
feats = ["area", "kitchen_area"] + geo_cat_feats + cat_feats

check_log_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(3.253588092155931, 0.31509825731129065)

# Breadcrumbs 


Posunęliśmy się trochę dalej, to cieszy. Tutaj możesz jeszcze trochę pogrzebać - w `geo_block`, ale zostawię to dla Ciebie.  Chcę Ci pomóc jeszcze "ugryźć" `breadcrumbs` 


In [138]:
breadcrumbs = (
    df["breadcrumbs"]
        .map(lambda x: {"breadcrumbs_{}".format(idx):val for idx,val in enumerate(x) })
        .apply(pd.Series)
)

breadcrumbs

,breadcrumbs_0,breadcrumbs_1,breadcrumbs_2,breadcrumbs_3,breadcrumbs_4,breadcrumbs_5,breadcrumbs_6,breadcrumbs_7
0,Москва,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN,NaN,NaN,NaN
1,Москва,Ховрино,м. Ховрино,МЦК Коптево,NaN,NaN,NaN,NaN
2,Москва,МЦК Кутузовская,ул. Лобачевского,NaN,NaN,NaN,NaN,NaN
3,Москва,Ховрино,м. Ховрино,МЦК Коптево,ул. Дыбенко,NaN,NaN,NaN
4,Москва,Даниловский,м. Автозаводская,МЦК Автозаводская,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11443,Москва,МЦК Кутузовская,NaN,NaN,NaN,NaN,NaN,NaN
11444,Москва,Хорошёво-Мнёвники,м. Октябрьское поле,МЦК Зорге,NaN,NaN,NaN,NaN
11445,Москва,Ховрино,м. Ховрино,МЦК Коптево,NaN,NaN,NaN,NaN
11446,Москва,Новая Москва,п. Внуковское,МЦК Кутузовская,NaN,NaN,NaN,NaN


In [119]:
df['breadcrumbs']

0          [Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]
1               [Москва, Ховрино, м. Ховрино, МЦК Коптево]
2              [Москва, МЦК Кутузовская, ул. Лобачевского]
3        [Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....
4        [Москва, Даниловский, м. Автозаводская, МЦК Ав...
                               ...                        
11443                            [Москва, МЦК Кутузовская]
11444    [Москва, Хорошёво-Мнёвники, м. Октябрьское пол...
11445           [Москва, Ховрино, м. Ховрино, МЦК Коптево]
11446    [Москва, Новая Москва, п. Внуковское, МЦК Куту...
11447    [Москва, Ярославский, МЦК Ростокино, ул. Красн...
Name: breadcrumbs, Length: 34180, dtype: object

Tutaj okazuje się, że maksymalna głębokość wynosi 7 (w przeciwieństwie do `geo_block`). Zobaczmy, co jeszcze jest w środku.


In [20]:
for feat in breadcrumbs.columns:
    print(feat)
    print(breadcrumbs[feat].value_counts().head(20))
    print("="*100)
    print()
    

    df["{}_cat".format(feat)] = breadcrumbs[feat].factorize()[0]

breadcrumbs_0
Москва    34180
Name: breadcrumbs_0, dtype: int64

breadcrumbs_1
Новая Москва              4531
МЦК Лужники               1245
МЦК Кутузовская           1237
Хорошёво-Мнёвники         1142
Филёвский Парк             945
МЦК Крымская               932
МЦК Шелепиха               842
Ховрино                    829
Раменки                    811
МЦК Коптево                800
Люблино                    666
МЦК Хорошёво               633
Некрасовка                 608
МЦК Нижегородская          564
МЦК Лихоборы               549
Кунцево                    537
Орехово-Борисово Южное     534
Выхино-Жулебино            494
Зябликово                  492
МЦК Ростокино              453
Name: breadcrumbs_1, dtype: int64

breadcrumbs_2
п. Сосенское             2522
п. Московский             919
м. Октябрьское поле       864
м. Выхино                 856
м. Ховрино                829
м. Братиславская          649
м. Рязанский проспект     600
м. Пятницкое шоссе        558
м. Фили     

In [121]:
for feat in breadcrumbs.columns:
    df["{}_metro".format(feat)] = breadcrumbs[feat]

df.head(3)

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,Тип объекта:,Количество корпусов:,Тип объявления:,Застройщик:,Общая площадь:,Дата публикации:,Количество комнат:,Парковка:,Дата обновления:,Количество этажей:,Сдача:,Комиссия агенту:,Высота потолков:,Этаж:,Этап строительства:,Новостройка:,Количество квартир:,Класс жилья:,Адрес:,Лифт:,Вид из окна:,Возможна ипотека:,Год постройки:,Этажность:,Мусоропровод:,Ремонт:,Площадь кухни:,Жилая комната:,Тип санузла:,Тип балкона:,Мебель на кухне:,Холодильник:,Интернет:,Мебель:,Телефон:,Свободная планировка:,Управляющая компания:,Количество подъездов:,Тип дома:,Тип комнат:,Покрытие пола:,Серия:,Тип перекрытия:,Возможен торг:,Класс:,Телевизор:,Тип фундамента:,Количество жилых помещений:,Детская площадка:,Стиральная машина:,Наибольшее количество этажей:,Наименьшее количество этажей:,Предоплата:,Охрана:_cat,Тип здания:_cat,Тип объекта:_cat,Количество корпусов:_cat,Тип объявления:_cat,Застройщик:_cat,Общая площадь:_cat,Дата публикации:_cat,Количество комнат:_cat,Парковка:_cat,Дата обновления:_cat,Количество этажей:_cat,Сдача:_cat,Комиссия агенту:_cat,Высота потолков:_cat,Этаж:_cat,Этап строительства:_cat,Новостройка:_cat,Количество квартир:_cat,Класс жилья:_cat,Адрес:_cat,Лифт:_cat,Вид из окна:_cat,Возможна ипотека:_cat,Год постройки:_cat,Этажность:_cat,Мусоропровод:_cat,Ремонт:_cat,Площадь кухни:_cat,Жилая комната:_cat,Тип санузла:_cat,Тип балкона:_cat,Мебель на кухне:_cat,Холодильник:_cat,Интернет:_cat,Мебель:_cat,Телефон:_cat,Свободная планировка:_cat,Управляющая компания:_cat,Количество подъездов:_cat,Тип дома:_cat,Тип комнат:_cat,Покрытие пола:_cat,Серия:_cat,Тип перекрытия:_cat,Возможен торг:_cat,Класс:_cat,Телевизор:_cat,Тип фундамента:_cat,Количество жилых помещений:_cat,Детская площадка:_cat,Стиральная машина:_cat,Наибольшее количество этажей:_cat,Наименьшее количество этажей:_cat,Предоплата:_cat,area,kitchen_area,geo_block_norm,geo_block_0_cat,geo_block_1_cat,geo_block_2_cat,geo_block_3_cat,geo_block_4_cat,geo_block_5_cat,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat,metro,pietro,l_pieter,breadcrumbs_0_metro,breadcrumbs_1_metro,breadcrumbs_2_metro,breadcrumbs_3_metro,breadcrumbs_4_metro,breadcrumbs_5_metro,breadcrumbs_6_metro,breadcrumbs_7_metro
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,квартиры,1,в новостройке,Доходный дом,54.3 м²,22 апреля,2,"отдельная многоуровневая, гостевая",18 мая,11,1 кв. 2020 года,без комиссии,2.8 м,8/11,Возведение стен,"Апарт-комплекс Nord, м. Алтуфьево",163,Комфорт класс,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54.3,-1.0,"[г. Москва, Лианозово]",0,0,-1,-1,-1,-1,0,0,0,0,-1,-1,-1,-1,NaN,8,11,Москва,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN,NaN,NaN,NaN
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,квартира,5,в новостройке,MR Group,38 м²,10 мая,2,подземная,16 мая,31,2 кв. 2021 года,без комиссии,3.15 м,6/31,Возведение стен,"ЖК Discovery (ЖК «Дискавери»), м. Ховрино",1251,Комфорт класс,,да,на улицу,да,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.0,-1.0,"[г. Москва, Ховрино]",0,1,-1,-1,-1,-1,0,1,1,1,-1,-1,-1,-1,NaN,6,31,Москва,Хов

Możesz zobaczyć wiele przydatnych informacji - na przykład nazwę stacji metra.


### `МЦК` - to jest skrót od  **Московское центральное кольцо** (ang. Moscow Central Circle) 

![](https://img.gazeta.ru/files3/393/10190393/TASS_17572950-pic905-895x505-29643.jpg)

In [21]:
breadcrumbs_cat_feats = [x for x in df.columns if "breadcrumbs" in x and "_cat" in x]
breadcrumbs_cat_feats

['breadcrumbs_0_cat',
 'breadcrumbs_1_cat',
 'breadcrumbs_2_cat',
 'breadcrumbs_3_cat',
 'breadcrumbs_4_cat',
 'breadcrumbs_5_cat',
 'breadcrumbs_6_cat',
 'breadcrumbs_7_cat']

Dodajemy 7 cech z `breadcrumbs` do tych wszystkich, które już mieliśmy  i przeliczamy. 


In [22]:
feats = ["area", "kitchen_area"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats

check_log_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(3.164857690608772, 0.2704769525269992)

Udało nam się ulepszyć wynik! 

## Rok budowy

In [34]:
df['Год постройки:']=df['Год постройки:'].astype('int')
df['Год постройки:'].head()

0      -1
1      -1
2    2021
3      -1
4      -1
Name: Год постройки:, dtype: int64

In [39]:
#Sprawdzamy model z dodanym rokiem budowy
feats = ["area", "kitchen_area", "Год постройки:"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
model= xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0)
check_log_model(df, feats, model)

(3.1644592194931884,
 0.2577625083062455,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [42]:
# Jest nieco lepiej. Dodaję zmienną do modelu na końcu notebooka.
eli5.show_weights(model, feature_names=feats)

Weight,Feature
0.2694,area
0.2079,Класс жилья:_cat
0.0903,geo_block_0_cat
0.0568,Ремонт:_cat
0.0172,Количество квартир:_cat
0.0170,Парковка:_cat
0.0154,Лифт:_cat
0.0147,breadcrumbs_4_cat
0.0137,Тип балкона:_cat
0.0132,Тип здания:_cat


# Liczba mieszkań

In [54]:
# Liczba mieszkań jest przechowywana jako object, konwertuję na int
df['Количество квартир:']=df['Количество квартир:'].astype('int')

In [55]:
#Sprawdzamy model z dodaną ilością mieszkań w formie numerycznej
feats = ["area", "kitchen_area", "Год постройки:", "Количество квартир:"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
model= xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0)
check_log_model(df, feats, model)

(3.0908824082760593,
 0.32609881269514573,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [56]:
# Wynik lepszy, dodaję zmienną do docelowego modelu.
eli5.show_weights(model, feature_names=feats)

Weight,Feature
0.2767,area
0.2026,Класс жилья:_cat
0.1055,geo_block_0_cat
0.0439,Ремонт:_cat
0.0222,Количество квартир:_cat
0.0195,Количество квартир:
0.0183,Тип балкона:_cat
0.0148,Высота потолков:_cat
0.0147,Количество корпусов:_cat
0.0146,Этажность:_cat


# Stacja metra

Czy lokal jest przy stacji metra?
Tworzę zmienną, która informuje, czy lokal znajduje się przy stacji metra



In [150]:
breadcrumbs=breadcrumbs.fillna('')
#for column in breadcrumbs.columns:
breadcrumbs['metro_tak_1'] = breadcrumbs["breadcrumbs_1"].str.contains('МЦК')
breadcrumbs['metro_tak_2'] = breadcrumbs["breadcrumbs_2"].str.contains('МЦК')
breadcrumbs['metro_tak_3'] = breadcrumbs["breadcrumbs_3"].str.contains('МЦК')
breadcrumbs['metro_tak_4'] = breadcrumbs["breadcrumbs_4"].str.contains('МЦК')
breadcrumbs['metro_tak_5'] = breadcrumbs["breadcrumbs_5"].str.contains('МЦК')
breadcrumbs['metro_tak_6'] = breadcrumbs["breadcrumbs_6"].str.contains('МЦК')


In [151]:
breadcrumbs['metro']=breadcrumbs.select_dtypes(include=['bool']).sum(axis=1)
breadcrumbs['metro'].value_counts()

1    33246
0      934
Name: metro, dtype: int64

In [176]:
breadcrumbs['new']=breadcrumbs['metro'].copy()


In [183]:
breadcrumbs.head()

,breadcrumbs_0,breadcrumbs_1,breadcrumbs_2,breadcrumbs_3,breadcrumbs_4,breadcrumbs_5,breadcrumbs_6,breadcrumbs_7,metro_tak_1,metro_tak_2,metro_tak_3,metro_tak_4,metro_tak_5,metro_tak_6,metro,new,New
0,Москва,Лианозово,м. Алтуфьево,МЦК Лихоборы,,,,,False,False,True,False,False,False,1,1,NaN
1,Москва,Ховрино,м. Ховрино,МЦК Коптево,,,,,False,False,True,False,False,False,1,1,NaN
2,Москва,МЦК Кутузовская,ул. Лобачевского,,,,,,True,False,False,False,False,False,1,1,1.0
3,Москва,Ховрино,м. Ховрино,МЦК Коптево,ул. Дыбенко,,,,False,False,True,False,False,False,1,1,NaN
4,Москва,Даниловский,м. Автозаводская,МЦК Автозаводская,,,,,False,False,True,False,False,False,1,1,NaN


In [184]:
#Chciałam zakodować ręcznie nazwy stacji metra, ale wyszło coś takiego :-)
#Są one umieszczone w różnych kolumnach tabeli.
#Po kolei filtruję każdą kolumnę i wpisuję numer jeśli w tej kolumnie jest nazwa stacji metra.

breadcrumbs.loc[(breadcrumbs.metro_tak_1 == True),'New']=1
breadcrumbs.loc[(breadcrumbs.metro_tak_2 == True),'New']=2
breadcrumbs.loc[(breadcrumbs.metro_tak_3 == True),'New']=3
breadcrumbs.loc[(breadcrumbs.metro_tak_4 == True),'New']=4
breadcrumbs.loc[(breadcrumbs.metro_tak_5 == True),'New']=5
breadcrumbs.loc[(breadcrumbs.metro_tak_6 == True),'New']=6
breadcrumbs['New'].value_counts()

3.0    16449
1.0    11880
6.0     2122
4.0     1650
2.0     1076
5.0       69
Name: New, dtype: int64

In [ ]:


breadcrumbs['metro_tak_1'] = breadcrumbs["breadcrumbs_1"].str.contains('МЦК')
breadcrumbs['metro_tak_2'] = breadcrumbs["breadcrumbs_2"].str.contains('МЦК')
breadcrumbs['metro_tak_3'] = breadcrumbs["breadcrumbs_3"].str.contains('МЦК')
breadcrumbs['metro_tak_4'] = breadcrumbs["breadcrumbs_4"].str.contains('МЦК')
breadcrumbs['metro_tak_5'] = breadcrumbs["breadcrumbs_5"].str.contains('МЦК')
breadcrumbs['metro_tak_6'] = breadcrumbs["breadcrumbs_6"].str.contains('МЦК')


In [154]:
df['metro']=breadcrumbs['metro']
df.head(3)

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,Тип объекта:,Количество корпусов:,Тип объявления:,Застройщик:,Общая площадь:,Дата публикации:,Количество комнат:,Парковка:,Дата обновления:,Количество этажей:,Сдача:,Комиссия агенту:,Высота потолков:,Этаж:,Этап строительства:,Новостройка:,Количество квартир:,Класс жилья:,Адрес:,Лифт:,Вид из окна:,Возможна ипотека:,Год постройки:,Этажность:,Мусоропровод:,Ремонт:,Площадь кухни:,Жилая комната:,Тип санузла:,Тип балкона:,Мебель на кухне:,Холодильник:,Интернет:,Мебель:,Телефон:,Свободная планировка:,Управляющая компания:,Количество подъездов:,Тип дома:,Тип комнат:,Покрытие пола:,Серия:,Тип перекрытия:,Возможен торг:,Класс:,Телевизор:,Тип фундамента:,Количество жилых помещений:,Детская площадка:,Стиральная машина:,Наибольшее количество этажей:,Наименьшее количество этажей:,Предоплата:,Охрана:_cat,Тип здания:_cat,Тип объекта:_cat,Количество корпусов:_cat,Тип объявления:_cat,Застройщик:_cat,Общая площадь:_cat,Дата публикации:_cat,Количество комнат:_cat,Парковка:_cat,Дата обновления:_cat,Количество этажей:_cat,Сдача:_cat,Комиссия агенту:_cat,Высота потолков:_cat,Этаж:_cat,Этап строительства:_cat,Новостройка:_cat,Количество квартир:_cat,Класс жилья:_cat,Адрес:_cat,Лифт:_cat,Вид из окна:_cat,Возможна ипотека:_cat,Год постройки:_cat,Этажность:_cat,Мусоропровод:_cat,Ремонт:_cat,Площадь кухни:_cat,Жилая комната:_cat,Тип санузла:_cat,Тип балкона:_cat,Мебель на кухне:_cat,Холодильник:_cat,Интернет:_cat,Мебель:_cat,Телефон:_cat,Свободная планировка:_cat,Управляющая компания:_cat,Количество подъездов:_cat,Тип дома:_cat,Тип комнат:_cat,Покрытие пола:_cat,Серия:_cat,Тип перекрытия:_cat,Возможен торг:_cat,Класс:_cat,Телевизор:_cat,Тип фундамента:_cat,Количество жилых помещений:_cat,Детская площадка:_cat,Стиральная машина:_cat,Наибольшее количество этажей:_cat,Наименьшее количество этажей:_cat,Предоплата:_cat,area,kitchen_area,geo_block_norm,geo_block_0_cat,geo_block_1_cat,geo_block_2_cat,geo_block_3_cat,geo_block_4_cat,geo_block_5_cat,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat,metro,pietro,l_pieter,breadcrumbs_0_metro,breadcrumbs_1_metro,breadcrumbs_2_metro,breadcrumbs_3_metro,breadcrumbs_4_metro,breadcrumbs_5_metro,breadcrumbs_6_metro,breadcrumbs_7_metro
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,квартиры,1,в новостройке,Доходный дом,54.3 м²,22 апреля,2,"отдельная многоуровневая, гостевая",18 мая,11,1 кв. 2020 года,без комиссии,2.8 м,8/11,Возведение стен,"Апарт-комплекс Nord, м. Алтуфьево",163,Комфорт класс,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54.3,-1.0,"[г. Москва, Лианозово]",0,0,-1,-1,-1,-1,0,0,0,0,-1,-1,-1,-1,1,8,11,Москва,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN,NaN,NaN,NaN
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,квартира,5,в новостройке,MR Group,38 м²,10 мая,2,подземная,16 мая,31,2 кв. 2021 года,без комиссии,3.15 м,6/31,Возведение стен,"ЖК Discovery (ЖК «Дискавери»), м. Ховрино",1251,Комфорт класс,,да,на улицу,да,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.0,-1.0,"[г. Москва, Ховрино]",0,1,-1,-1,-1,-1,0,1,1,1,-1,-1,-1,-1,1,6,31,Москва,Ховрино

In [149]:
nazwy_metra_1=[x for x in breadcrumbs["breadcrumbs_1"] if "МЦК" in x]
nazwy_metra_1

TypeError: 'numpy.ndarray' object is not callable

In [137]:
breadcrumbs[['metro_tak_1', 'metro_tak_2', 'metro_tak_3', 'metro_tak_4', 'metro_tak_5', 'metro_tak_6']]

,metro_tak_1,metro_tak_2,metro_tak_3,metro_tak_4,metro_tak_5,metro_tak_6
0,False,False,True,NaN,True,True
1,False,False,True,NaN,True,True
2,True,False,NaN,NaN,NaN,NaN
3,False,False,True,False,True,True
4,False,False,True,NaN,True,True
...,...,...,...,...,...,...
11443,True,NaN,NaN,NaN,NaN,NaN
11444,False,False,True,NaN,True,True
11445,False,False,True,NaN,True,True
11446,False,False,True,NaN,True,True


In [185]:
df['metro_stacja']=breadcrumbs['New']
df.head()

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,Тип объекта:,Количество корпусов:,Тип объявления:,Застройщик:,Общая площадь:,Дата публикации:,Количество комнат:,Парковка:,Дата обновления:,Количество этажей:,Сдача:,Комиссия агенту:,Высота потолков:,Этаж:,Этап строительства:,Новостройка:,Количество квартир:,Класс жилья:,Адрес:,Лифт:,Вид из окна:,Возможна ипотека:,Год постройки:,Этажность:,Мусоропровод:,Ремонт:,Площадь кухни:,Жилая комната:,Тип санузла:,Тип балкона:,Мебель на кухне:,Холодильник:,Интернет:,Мебель:,Телефон:,Свободная планировка:,Управляющая компания:,Количество подъездов:,Тип дома:,Тип комнат:,Покрытие пола:,Серия:,Тип перекрытия:,Возможен торг:,Класс:,Телевизор:,Тип фундамента:,Количество жилых помещений:,Детская площадка:,Стиральная машина:,Наибольшее количество этажей:,Наименьшее количество этажей:,Предоплата:,Охрана:_cat,Тип здания:_cat,Тип объекта:_cat,Количество корпусов:_cat,Тип объявления:_cat,Застройщик:_cat,Общая площадь:_cat,Дата публикации:_cat,Количество комнат:_cat,Парковка:_cat,Дата обновления:_cat,Количество этажей:_cat,Сдача:_cat,Комиссия агенту:_cat,Высота потолков:_cat,Этаж:_cat,Этап строительства:_cat,Новостройка:_cat,Количество квартир:_cat,Класс жилья:_cat,Адрес:_cat,Лифт:_cat,Вид из окна:_cat,Возможна ипотека:_cat,Год постройки:_cat,Этажность:_cat,Мусоропровод:_cat,Ремонт:_cat,Площадь кухни:_cat,Жилая комната:_cat,Тип санузла:_cat,Тип балкона:_cat,Мебель на кухне:_cat,Холодильник:_cat,Интернет:_cat,Мебель:_cat,Телефон:_cat,Свободная планировка:_cat,Управляющая компания:_cat,Количество подъездов:_cat,Тип дома:_cat,Тип комнат:_cat,Покрытие пола:_cat,Серия:_cat,Тип перекрытия:_cat,Возможен торг:_cat,Класс:_cat,Телевизор:_cat,Тип фундамента:_cat,Количество жилых помещений:_cat,Детская площадка:_cat,Стиральная машина:_cat,Наибольшее количество этажей:_cat,Наименьшее количество этажей:_cat,Предоплата:_cat,area,kitchen_area,geo_block_norm,geo_block_0_cat,geo_block_1_cat,geo_block_2_cat,geo_block_3_cat,geo_block_4_cat,geo_block_5_cat,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat,metro,pietro,l_pieter,breadcrumbs_0_metro,breadcrumbs_1_metro,breadcrumbs_2_metro,breadcrumbs_3_metro,breadcrumbs_4_metro,breadcrumbs_5_metro,breadcrumbs_6_metro,breadcrumbs_7_metro,metro_stacja
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,квартиры,1,в новостройке,Доходный дом,54.3 м²,22 апреля,2,"отдельная многоуровневая, гостевая",18 мая,11,1 кв. 2020 года,без комиссии,2.8 м,8/11,Возведение стен,"Апарт-комплекс Nord, м. Алтуфьево",163,Комфорт класс,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54.3,-1.0,"[г. Москва, Лианозово]",0,0,-1,-1,-1,-1,0,0,0,0,-1,-1,-1,-1,1,8,11,Москва,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN,NaN,NaN,NaN,3.0
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,квартира,5,в новостройке,MR Group,38 м²,10 мая,2,подземная,16 мая,31,2 кв. 2021 года,без комиссии,3.15 м,6/31,Возведение стен,"ЖК Discovery (ЖК «Дискавери»), м. Ховрино",1251,Комфорт класс,,да,на улицу,да,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.0,-1.0,"[г. Москва, Ховрино]",0,1,-1,-1,-1,-1,0,1,1,1,-1,-1,-1,-1,1,6,

# Które piętro

In [70]:
#Sprawdzamy piętro

df['Этаж:'].value_counts()

2        658
3        485
4        446
6        419
8        403
        ... 
16/59      1
33/77      1
24/82      1
28/42      1
24/34      1
Name: Этаж:, Length: 1241, dtype: int64

In [71]:
def pietro(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return int(val.split("/")[0])

df["pietro"] = df["Этаж:"].map(pietro)

# Liczba pokoi

In [72]:
df["pietro"].value_counts()

2     2810
3     2184
4     1956
5     1850
6     1769
      ... 
77       2
67       1
60       1
97       1
83       1
Name: pietro, Length: 90, dtype: int64

In [97]:
df['Количество комнат:']=df['Количество комнат:'].astype('int')

In [98]:
#sprawdzam model 
feats = ["area", "kitchen_area", "Год постройки:", "Количество квартир:","pietro",  "Количество комнат:"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
model= xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0)
check_log_model(df, feats, model)


(3.04912254055789,
 0.31681681224337593,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

# Liczba pięter w budynku

In [99]:
#liczba pięter w budynku 


df["l_pieter"] = df["Количество этажей:"].astype('int')

In [100]:
#sprawdzam model 
feats = ["area", "kitchen_area", "Год постройки:", "Количество квартир:","pietro", "l_pieter", "Количество комнат:"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
model= xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0)
check_log_model(df, feats, model)

(2.983769135046969,
 0.3118431314787995,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [112]:
#sprawdzam model - hiperparametry
feats = ["area", "kitchen_area", "Год постройки:", "Количество квартир:","pietro", "l_pieter", "Количество комнат:"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
model= xgb.XGBRegressor(learning_rate = 0.1, max_depth= 7, n_estimators= 200, random_state= 0)
check_log_model(df, feats, model)

(2.6359270187339447,
 0.27764476686122636,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [157]:
#sprawdzam model - z metrem
feats = ["area", "kitchen_area", "Год постройки:", "Количество квартир:","pietro", "l_pieter", "Количество комнат:", "metro"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
model= xgb.XGBRegressor(learning_rate = 0.1, max_depth= 7, n_estimators= 400, random_state= 0)
check_log_model(df, feats, model)

(2.5319185512256217,
 0.2861067734685343,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [92]:
pd.set_option('display.max_columns', None)
df.head(3)

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,Тип объекта:,Количество корпусов:,Тип объявления:,Застройщик:,Общая площадь:,Дата публикации:,Количество комнат:,Парковка:,Дата обновления:,Количество этажей:,Сдача:,Комиссия агенту:,Высота потолков:,Этаж:,Этап строительства:,Новостройка:,Количество квартир:,Класс жилья:,Адрес:,Лифт:,Вид из окна:,Возможна ипотека:,Год постройки:,Этажность:,Мусоропровод:,Ремонт:,Площадь кухни:,Жилая комната:,Тип санузла:,Тип балкона:,Мебель на кухне:,Холодильник:,Интернет:,Мебель:,Телефон:,Свободная планировка:,Управляющая компания:,Количество подъездов:,Тип дома:,Тип комнат:,Покрытие пола:,Серия:,Тип перекрытия:,Возможен торг:,Класс:,Телевизор:,Тип фундамента:,Количество жилых помещений:,Детская площадка:,Стиральная машина:,Наибольшее количество этажей:,Наименьшее количество этажей:,Предоплата:,Охрана:_cat,Тип здания:_cat,Тип объекта:_cat,Количество корпусов:_cat,Тип объявления:_cat,Застройщик:_cat,Общая площадь:_cat,Дата публикации:_cat,Количество комнат:_cat,Парковка:_cat,Дата обновления:_cat,Количество этажей:_cat,Сдача:_cat,Комиссия агенту:_cat,Высота потолков:_cat,Этаж:_cat,Этап строительства:_cat,Новостройка:_cat,Количество квартир:_cat,Класс жилья:_cat,Адрес:_cat,Лифт:_cat,Вид из окна:_cat,Возможна ипотека:_cat,Год постройки:_cat,Этажность:_cat,Мусоропровод:_cat,Ремонт:_cat,Площадь кухни:_cat,Жилая комната:_cat,Тип санузла:_cat,Тип балкона:_cat,Мебель на кухне:_cat,Холодильник:_cat,Интернет:_cat,Мебель:_cat,Телефон:_cat,Свободная планировка:_cat,Управляющая компания:_cat,Количество подъездов:_cat,Тип дома:_cat,Тип комнат:_cat,Покрытие пола:_cat,Серия:_cat,Тип перекрытия:_cat,Возможен торг:_cat,Класс:_cat,Телевизор:_cat,Тип фундамента:_cat,Количество жилых помещений:_cat,Детская площадка:_cat,Стиральная машина:_cat,Наибольшее количество этажей:_cat,Наименьшее количество этажей:_cat,Предоплата:_cat,area,kitchen_area,geo_block_norm,geo_block_0_cat,geo_block_1_cat,geo_block_2_cat,geo_block_3_cat,geo_block_4_cat,geo_block_5_cat,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat,metro,pietro
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,квартиры,1,в новостройке,Доходный дом,54.3 м²,22 апреля,2,"отдельная многоуровневая, гостевая",18 мая,11,1 кв. 2020 года,без комиссии,2.8 м,8/11,Возведение стен,"Апарт-комплекс Nord, м. Алтуфьево",163,Комфорт класс,,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54.3,-1.0,"[г. Москва, Лианозово]",0,0,-1,-1,-1,-1,0,0,0,0,-1,-1,-1,-1,NaN,8
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,квартира,5,в новостройке,MR Group,38 м²,10 мая,2,подземная,16 мая,31,2 кв. 2021 года,без комиссии,3.15 м,6/31,Возведение стен,"ЖК Discovery (ЖК «Дискавери»), м. Ховрино",1251,Комфорт класс,,да,на улицу,да,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.0,-1.0,"[г. Москва, Ховрино]",0,1,-1,-1,-1,-1,0,1,1,1,-1,-1,-1,-1,NaN,6
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 ма

## Kaggle submit
Tworzymy plik do wgrania do Kaggle :) 

In [188]:
feats = ["area", "kitchen_area", "Год постройки:", "Количество квартир:","pietro", "l_pieter", "Количество комнат:", "metro_stacja", "metro"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats
df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()

X_train = df_train[feats]
y_train = df_train["price"]
y_log_train = np.log(y_train)

X_test = df_test[feats]

#Strojenie hiperparametrów - intuicyjnie
#model = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0)
#model= xgb.XGBRegressor(learning_rate = 0.1, max_depth= 7, n_estimators= 200, random_state= 0)

model= xgb.XGBRegressor(learning_rate = 0.1, max_depth= 9, n_estimators= 500, random_state= 0)
model.fit(X_train, y_log_train)
y_log_pred = model.predict(X_test)
y_pred = np.exp(y_log_pred)


df_test["price"] = y_pred
df_test[ ["id", "price"] ].to_csv("../output/xgb_all.csv", index=False)